In [1]:
# we'll use coral to manage the sequence data, and pandas for loading the csv

import coral as cor
import pandas as pd

In [11]:
# location of the plasmid sequence file
target_plasmid = './plasmids/pMJS1AC_nth6_egf2.gb'
donor_plasmid = './plasmids/T7 controller plasmid.gb'

# location of the plasmid sequence file
t7_plasmid = cor.seqio.read_dna(donor_plasmid)
gfp_plasmid = cor.seqio.read_dna(target_plasmid)

In [12]:
# this is our target plasmid -> where we want the new resistance gene
gfp_plasmid.display()

In [13]:
# It has these features defined in the genbank file
gfp_plasmid.features

[BsaI ggtctc (1/5) 'misc_feature' feature (83 to 89) on strand 0,
 scar(4) 'misc_feature' feature (90 to 94) on strand 0,
 LacP +RBS 'misc_feature' feature (100 to 232) on strand 0,
 lacP start 'misc_feature' feature (100 to 106) on strand 0,
 New Feature 'misc_feature' feature (226 to 232) on strand 0,
 open reading frame 1 'misc_feature' feature (238 to 1016) on strand 0,
 scar(1) 'misc_feature' feature (421 to 425) on strand 0,
 scar 'misc_feature' feature (632 to 636) on strand 0,
 scar(2) 'misc_feature' feature (823 to 827) on strand 0,
 stop codon 'misc_feature' feature (1015 to 1016) on strand 0,
 scar(3) 'misc_feature' feature (1038 to 1042) on strand 0,
 BsaI ggtctc (1/5)(1) 'misc_feature' feature (1043 to 1049) on strand 1,
 ColE1\origin 'rep_origin' feature (1207 to 1890) on strand 1,
 ColE1 origin 'rep_origin' feature (1207 to 1890) on strand 1,
 Kanamycin\phosphotransferase 'CDS' feature (1989 to 2805) on strand 1]

In [14]:
# let's grab the kanamycin feature
kan_feature = gfp_plasmid.features[-1]
kan_feature

Kanamycin\phosphotransferase 'CDS' feature (1989 to 2805) on strand 1

In [15]:
# and create a new plasmid with it excised
gfp_plasmid_without_kan = gfp_plasmid[0:kan_feature.start] + gfp_plasmid[kan_feature.stop::]
gfp_plasmid_without_kan.circularize().display()

In [18]:
# this is our donor plasmid. it has a gene that codes for strep resistance
t7_plasmid.display()

In [19]:
# let's grab the strep resistance gene from the donor plasmid
strep_feature = t7_plasmid.features[-2]
strep_sequence = t7_plasmid.extract(strep_feature)

In [21]:
# and put it into the target plasmid (note, we're using the original sequence info to get the original kan loocation)
# Note: we have to insert the RC form of the sequence to get it to align with the read direction of the original 
# resistance gene. This means we'll also have to be careful when create overlaps to make sure they overlap on the correct sides

final_plasmid = gfp_plasmid[0:kan_feature.start] + strep_sequence.reverse_complement() + gfp_plasmid[kan_feature.stop::]
final_plasmid.circularize().display()

In [ ]:
# great. This is what we want our plasmid to look like when we're done.

In [25]:
final_plasmid.features

[BsaI ggtctc (1/5) 'misc_feature' feature (83 to 89) on strand 0,
 scar(4) 'misc_feature' feature (90 to 94) on strand 0,
 LacP +RBS 'misc_feature' feature (100 to 232) on strand 0,
 lacP start 'misc_feature' feature (100 to 106) on strand 0,
 New Feature 'misc_feature' feature (226 to 232) on strand 0,
 open reading frame 1 'misc_feature' feature (238 to 1016) on strand 0,
 scar(1) 'misc_feature' feature (421 to 425) on strand 0,
 scar 'misc_feature' feature (632 to 636) on strand 0,
 scar(2) 'misc_feature' feature (823 to 827) on strand 0,
 stop codon 'misc_feature' feature (1015 to 1016) on strand 0,
 scar(3) 'misc_feature' feature (1038 to 1042) on strand 0,
 BsaI ggtctc (1/5)(1) 'misc_feature' feature (1043 to 1049) on strand 1,
 ColE1\origin 'rep_origin' feature (1207 to 1890) on strand 1,
 ColE1 origin 'rep_origin' feature (1207 to 1890) on strand 1,
 Streptomycin\adenylyltransferase 'CDS' feature (1989 to 2781) on strand 1,
 biobrick\sequence 'misc_feature' feature (12419 to 27

In [27]:
# To build primers for Gibson assembly, we need to 20bp overlap before and after the inserted gene

left_overlap_start = 1989 - 20
right_overlap_start = 2781 + 20

# but we actually roll over to the other end of the plasmid

from_zero_index = right_overlap_start - len(final_plasmid)

In [28]:
primer_a1 = final_plasmid[2781::] + final_plasmid[0:from_zero_index]
primer_b1 = final_plasmid[left_overlap_start:1989]

In [30]:
primer_a1.top

AACACCCCTTGTATTACTGT

In [31]:
primer_b1.bottom

GGTAACTGTCAGACCAAGTT

In [32]:
# great. these primers will excise the kan gene, now let's build the primers for the strep gene
# ... this is actually quite easy

In [54]:
primer_a1comp = cor.design.primer(strep_sequence, overhang=primer_a1.flip(), min_len=20)
primer_b1comp = cor.design.primer(strep_sequence.reverse_complement(), overhang=primer_b1, min_len=20)

In [56]:
primer_a1comp

Primer: acagtaatacaaggggtgttATGAGGGAAGCGGTGATCGC Tm: 70.97

In [57]:
primer_b1comp

Primer: aacttggtctgacagttaccTTATTTGCCGACTACCTTGGTG Tm: 65.03

In [59]:
# that's it. Add these primers + Gibson mastermix + plasmids and TA'DA! you'll end up with this
final_plasmid.circularize().display()